In [1]:
import pandas as pd
import numpy as np
#pd.options.display.float_format = '{:.8f}'.format

In [80]:
dummiesdF = pd.read_csv("sales_data.csv")
dummiesdF = pd.get_dummies(dummiesdF)
labels = dummiesdF.iloc[0:,3]
dummiesdF = dummiesdF.drop(['flag_N','flag_Y'], axis = 1)
dummiesdF

,house_val,car_prob,gender_F,gender_M,gender_U,education_0. lessHS,education_1. HS,education_2. Some College,education_3. Bach,education_4. Grad,...,fam_income_D,fam_income_E,fam_income_F,fam_income_G,fam_income_H,fam_income_I,fam_income_J,fam_income_K,fam_income_L,fam_income_U
0,756460,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,213171,3,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,111147,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,354151,2,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,117087,7,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
5,248694,1,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
6,2000000,5,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,416925,2,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8,207676,5,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,241380,6,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [81]:
# Rescaling the house valuation
m, M = dummiesdF.house_val.min(), dummiesdF.house_val.max()
dummiesdF.house_val
dummiesdF['house_val'] = (dummiesdF.house_val - m) / (M - m)

# Rescaling the probability of buying a car (column 1)
dummiesdF['car_prob'] = dummiesdF.car_prob / 10 
dummiesdF.head(10)

,house_val,car_prob,gender_F,gender_M,gender_U,education_0. lessHS,education_1. HS,education_2. Some College,education_3. Bach,education_4. Grad,...,fam_income_D,fam_income_E,fam_income_F,fam_income_G,fam_income_H,fam_income_I,fam_income_J,fam_income_K,fam_income_L,fam_income_U
0,0.075646,0.1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.021317,0.3,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,0.011115,0.1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.035415,0.2,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.011709,0.7,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0.024869,0.1,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
6,0.200000,0.5,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0.041693,0.2,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8,0.020768,0.5,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,0.024138,0.6,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [82]:
from sklearn.decomposition import PCA
pca = PCA(.90)
pca.fit(dummiesdF.values)
print(pca.explained_variance_ratio_)
train_data = pca.transform(dummiesdF.values)
#print(train_data[:5,0:], len(train_data),len(train_data[0]))

[0.1018425  0.06875614 0.06309487 0.04545772 0.04386652 0.04100509
 0.03882016 0.03404133 0.03313232 0.0304234  0.02849785 0.02796583
 0.02614965 0.02478551 0.02364427 0.02344028 0.02175069 0.02109143
 0.02034987 0.01957887 0.01862273 0.01770032 0.01736481 0.01570375
 0.01559861 0.01449556 0.01262651 0.01200118 0.01062314 0.01056973
 0.00891777 0.00829892]


In [83]:
from sklearn.naive_bayes import BernoulliNB

NBBclassifier = BernoulliNB()
NBBclassifier.fit(train_data, labels)
print(NBBclassifier.score(train_data,labels))

from sklearn.naive_bayes import GaussianNB

NBGclassifier = GaussianNB()
NBGclassifier.fit(train_data, labels)
print(NBGclassifier.score(train_data,labels))

0.660275
0.665125


In [59]:
from logisticReg import logistic_regression
logit_reg = logistic_regression()

logit_reg.fit(train_data, labels, 0)
print("Score sobre el conjunto de entrenamiento:", logit_reg.score(train_data, labels))
#print("Score sobre el conjunto de validación:", logit_reg.score(test_data, labels))

Score sobre el conjunto de entrenamiento: 0.6159


In [84]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs', max_iter=200)
logisticRegr.fit(train_data, labels)
logisticRegr.score(train_data,labels)

0.686225

In [62]:
from sklearn.svm import SVC
SVMclassifier = SVC(gamma='auto')
SVMclassifier.fit(train_data, labels) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [63]:
SVMclassifier.score(train_data,labels)

0.697225

In [90]:
dropdF = pd.read_csv("sales_data.csv").dropna()
dropdF = pd.get_dummies(dropdF)
droplabels = dropdF.iloc[0:,3]
dropdF = dropdF.drop(['flag_N','flag_Y'], axis = 1)
dropdF
# Rescaling the house valuation
m, M = dropdF.house_val.min(), dropdF.house_val.max()
dropdF.house_val
dropdF['house_val'] = (dropdF.house_val - m) / (M - m)

# Rescaling the probability of buying a car (column 1)
dropdF['car_prob'] = dropdF.car_prob / 10 
dropdF.head(10)

,house_val,car_prob,gender_F,gender_M,gender_U,education_0. lessHS,education_1. HS,education_2. Some College,education_3. Bach,education_4. Grad,...,fam_income_D,fam_income_E,fam_income_F,fam_income_G,fam_income_H,fam_income_I,fam_income_J,fam_income_K,fam_income_L,fam_income_U
5,0.024869,0.1,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
7,0.041693,0.2,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
11,0.024569,0.3,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
12,0.036059,0.1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
15,0.016288,0.7,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0.000000,0.5,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
17,0.023956,0.3,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
18,0.006082,0.9,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0.000000,0.8,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
22,0.013673,0.4,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [91]:
from sklearn.decomposition import PCA
pca = PCA(.90)
pca.fit(dropdF.values)
print(pca.explained_variance_ratio_)
droptrain_data = pca.transform(dropdF.values)

from sklearn.naive_bayes import BernoulliNB
NBBclassifier = BernoulliNB()
NBBclassifier.fit(droptrain_data, droplabels)
print(NBBclassifier.score(droptrain_data,droplabels))

from sklearn.naive_bayes import GaussianNB

NBGclassifier = GaussianNB()
NBGclassifier.fit(droptrain_data, droplabels)
print(NBGclassifier.score(droptrain_data,droplabels))

[0.1152078  0.07408091 0.0636038  0.04945617 0.04398606 0.04211952
 0.03950844 0.03582999 0.03345005 0.03198506 0.02913542 0.02739627
 0.02603913 0.02574746 0.02474266 0.02451556 0.02200078 0.02085647
 0.02046326 0.0195934  0.01821077 0.01773959 0.01711359 0.01559665
 0.01438039 0.01272986 0.01172975 0.01133607 0.01020722 0.00917265]
0.6675014856948808
0.6712793955344257


In [95]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs', max_iter=200)
logisticRegr.fit(droptrain_data, droplabels)
print(logisticRegr.score(droptrain_data,droplabels))

from sklearn.svm import SVC
SVMclassifier = SVC(gamma='auto')
print(SVMclassifier.fit(droptrain_data, droplabels))

print(SVMclassifier.score(droptrain_data,droplabels))

0.6948807199252908
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.7073181084981747


In [96]:
SVMpolclassifier = SVC(gamma='auto', kernel='linear')
print(SVMpolclassifier.fit(droptrain_data, droplabels))

print(SVMpolclassifier.score(droptrain_data,droplabels))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.6929705407929366


In [ ]:
from sklearn.svm import SVC
SVMpolclassifier = SVC(gamma='auto', kernel='poly', degree=3)
print(SVMpolclassifier.fit(droptrain_data, droplabels))

print(SVMpolclassifier.score(droptrain_data,droplabels))